This notebook examines the use of various hyperparameters pertaining to guides, predictors, or the debiased training process.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pydebiaseddta.guides import BoWDTA, IDDTA
from pydebiaseddta.debiasing import DebiasedDTA
from pydebiaseddta.predictors import DeepDTA, BPEDTA, LMDTA
from pydebiaseddta.utils import load_sample_dta_data
from pydebiaseddta.evaluation import evaluate_predictions

guides = [BoWDTA, IDDTA]
predictors = [DeepDTA, BPEDTA]
train_ligands, train_proteins, train_labels = load_sample_dta_data(mini=True)["train"]
test_ligands, test_proteins, test_labels = load_sample_dta_data(mini=True)["test"]

In [2]:
for guide in guides:
    for predictor in predictors:
        print(guide.__name__, predictor.__name__)
        debiaseddta = DebiasedDTA(guide, predictor, predictor_params={"n_epochs": 2})
        train_hist = debiaseddta.train(train_ligands,
                                       train_proteins,
                                       train_labels,
                                       val_splits = {"cold_both": [test_ligands, test_proteins, test_labels]},
                                       metrics_tracked=["mae", "mse", "r2"])
        print(train_hist)
        preds = debiaseddta.predictor_instance.predict(test_ligands, test_proteins)
        scores = evaluate_predictions(test_labels, preds, metrics=["ci", "mse", "r2", "mae", "rmse"])   

BoWDTA DeepDTA
1/1 [==============================] - 0s 604us/step - loss: 42.8252 - mean_squared_error: 30.4498
{'train': {'mae': [5.376427, 4.969446], 'mse': [30.473136, 26.256895], 'r2': [-18.421735, -15.734558]}, 'val_splits': {'cold_both': {'mae': [5.148385, 4.741596], 'mse': [26.797692, 22.774888], 'r2': [-90.656661, -76.897386]}}}
BoWDTA BPEDTA
1/1 [==============================] - 0s 668us/step - loss: 43.0771 - mean_squared_error: 30.6801
{'train': {'mae': [5.397292, 5.025335], 'mse': [30.696363, 26.802896], 'r2': [-18.564006, -16.082546]}, 'val_splits': {'cold_both': {'mae': [5.171109, 4.812014], 'mse': [27.032222, 23.450272], 'r2': [-91.458828, -79.207416]}}}
IDDTA DeepDTA
1/1 [==============================] - 0s 564us/step - loss: 45.7621 - mean_squared_error: 30.4498
{'train': {'mae': [5.376427, 4.971248], 'mse': [30.473136, 26.274572], 'r2': [-18.421735, -15.745824]}, 'val_splits': {'cold_both': {'mae': [5.148385, 4.743513], 'mse': [26.797692, 22.792888], 'r2': [-90.65

In [8]:
debiaseddta = DebiasedDTA(BoWDTA, DeepDTA, predictor_params={
    "n_epochs": 100,
    "model_folder": "./temp/",
    "early_stopping_metric": "mae",
    "early_stopping_num_epochs": 3,
    "early_stopping_split": "val_set",
    "min_epochs": 15,
    "optimizer": "adam"})
train_hist = debiaseddta.train(train_ligands,
                                train_proteins,
                                train_labels,
                                val_splits = {"val_set": [test_ligands, test_proteins, test_labels]},
                                metrics_tracked=["mae", "mse", "r2"])
preds = debiaseddta.predictor_instance.predict(test_ligands, test_proteins)
scores = evaluate_predictions(test_labels, preds, metrics=["ci", "mse", "r2", "mae", "rmse"])   
print("MAE in val_set for last 5 epochs:", train_hist["val_splits"]["val_set"]["mae"][-5:])
print("MAE in val_set in the final model:", scores["mae"])

1/1 [==============================] - 0s 693us/step - loss: 3.0037 - mean_squared_error: 1.5563
Early stopping due to no increase to mae in val_set split for 3 epochs.
Retrieved the saved best model.
MAE in val_set for last 5 epochs: [0.630919, 0.417429, 0.494799, 0.47176, 0.453359]
MAE in val_set in the final model: 0.4174285904109863


In [9]:
debiaseddta = DebiasedDTA(IDDTA, BPEDTA, predictor_params={
    "n_epochs": 100,
    "model_folder": "./temp/",
    "early_stopping_metric": "mse",
    "early_stopping_metric_threshold": 1.6,
    "early_stopping_split": "train",
    "optimizer": "sgd",
    "learning_rate": 0.1})
train_hist = debiaseddta.train(train_ligands,
                                train_proteins,
                                train_labels,
                                val_splits = {"val_set": [test_ligands, test_proteins, test_labels]},
                                metrics_tracked=["mae", "mse", "r2"])
preds = debiaseddta.predictor_instance.predict(train_ligands, train_proteins)
scores = evaluate_predictions(train_labels, preds, metrics=["ci", "mse", "r2", "mae", "rmse"])    
print("MSE in train split for last 5 epochs:", train_hist["train"]["mse"][-5:])
print("MSE in train split in the final model:", scores["mse"])

1/1 [==============================] - 0s 585us/step - loss: 2.1554 - mean_squared_error: 1.6775
Early stopping training due to convergence on the train split.
MSE in train split for last 5 epochs: [14.559056, 8.229033, 3.157013, 1.702, 1.57221]
MSE in train split in the final model: 1.572210015864044


In [10]:
debiaseddta = DebiasedDTA(BoWDTA, BPEDTA, predictor_params={"n_epochs": 10}, guide_params={
    "max_depth": 4,
    "min_samples_split": 5,
    "min_samples_leaf": 3,
    "ligand_vector_mode": "freq",
    "prot_vector_mode": "binary",
    "vocab_size": "low",
    "criterion": "poisson",
    "input_rank": 10,
})
train_hist = debiaseddta.train(train_ligands, train_proteins, train_labels, metrics_tracked=["mae", "mse", "r2"])
preds = debiaseddta.predictor_instance.predict(test_ligands, test_proteins)
scores = evaluate_predictions(test_labels, preds, metrics=["ci", "mse", "r2", "mae", "rmse"])   

1/1 [==============================] - 0s 671us/step - loss: 7.3133 - mean_squared_error: 1.6780


In [11]:
debiaseddta = DebiasedDTA(IDDTA,
                          BPEDTA,
                          predictor_params={"n_epochs": 4},
                          guide_error_exponent=1,
                          weight_tempering_num_epochs=5,
                          weight_temperature=2,
                          weight_prior=0.01,
                          weight_rank_based=True
                          )
train_hist = debiaseddta.train(train_ligands,
                               train_proteins,
                               train_labels,
                               metrics_tracked=["mae", "mse", "r2"],
                               weights_save_path="./temp/exp.coef")
preds = debiaseddta.predictor_instance.predict(test_ligands, test_proteins)
scores = evaluate_predictions(test_labels, preds, metrics=["ci", "mse", "r2", "mae", "rmse"])   

1/1 [==============================] - 0s 731us/step - loss: 20.6351 - mean_squared_error: 19.0375


In [12]:
debiaseddta = DebiasedDTA(BoWDTA,
                          BPEDTA,
                          predictor_params={"n_epochs": 4},
                          guide_error_exponent=1,
                          weight_tempering_num_epochs=5,
                          weight_temperature=2,
                          weight_prior=0.01,
                          weight_rank_based=True
                          )
train_hist = debiaseddta.train(train_ligands,
                               train_proteins,
                               train_labels,
                               metrics_tracked=["mae", "mse", "r2"],
                               weights_load_path="./temp/exp.coef")
preds = debiaseddta.predictor_instance.predict(test_ligands, test_proteins)
scores = evaluate_predictions(test_labels, preds, metrics=["ci", "mse", "r2", "mae", "rmse"])   

1/1 [==============================] - 0s 655us/step - loss: 18.7288 - mean_squared_error: 19.0206
